# **FORECAST CREDITOS GENERALES | Queretaro**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "queretaro" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_queretaro',
 'df_financiamientos_2020_queretaro',
 'df_financiamientos_2021_queretaro',
 'df_financiamientos_2022_queretaro',
 'df_financiamientos_2023_queretaro',
 'df_financiamientos_2024_queretaro']

# **Forecast créditos generales (cantidad)**

### BASE

In [14]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [15]:
# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y contar el número de registros únicos
    total_creditos_df = df.groupby(['mes']).size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'num_registros']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_conteo_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_conteo_creditos_pachuca

,año,mes,num_registros
0,2019,1,556
1,2019,2,743
2,2019,3,774
3,2019,4,628
4,2019,5,610
...,...,...,...
61,2024,2,937
62,2024,3,916
63,2024,4,1003
64,2024,5,878


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [17]:
from sklearn.metrics import r2_score
import numpy as np

# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.0840213076640034
Error cuadrático medio (MSE): 17812.014163870263
Error absoluto medio (MAE): 117.85430792586529
Raíz del error cuadrático medio (RMSE): 133.4616580290769


In [18]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
nuevas_fechas.head()

total_conteo_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_conteo_pachuca

,año,mes,num_registros
0,2019,1,556
1,2019,2,743
2,2019,3,774
3,2019,4,628
4,2019,5,610
...,...,...,...
91,2026,8,792
92,2026,9,804
93,2026,10,816
94,2026,11,828


In [19]:
# Asegurarse de que la columna 'año' es de tipo entero
#total_conteo_pachuca['año'] = total_conteo_pachuca['año'].astype(int)

total_conteo_pachuca = total_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_conteo_pachuca['modalidad'] = 'Créditos (todos)'
#total_conteo_pachuca = total_conteo_pachuca.groupby('año').agg({'num_registros': 'sum', 'modalidad': 'first'}).reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales
# Formatear la columna 'montos' con la función personalizada
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].map(format_number)
total_conteo_pachuca = total_conteo_pachuca.iloc[1:].reset_index(drop=True)

total_conteo_pachuca

,año,num_registros,modalidad
0,2025,"9,264",Créditos (todos)
1,2026,"9,288",Créditos (todos)
2,2019,"8,676",Créditos (todos)
3,2020,"8,920",Créditos (todos)
4,2021,"10,112",Créditos (todos)
5,2022,"8,873",Créditos (todos)
6,2023,"8,364",Créditos (todos)
7,2024,"5,435",Créditos (todos)


In [20]:
# Valores conocidos
registros_2022 = 8873
registros_2023 = 8364

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))


Estimación de registros en 2024: 7884


In [21]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].replace({',': ''}, regex=True).astype(int)

# Ahora intenta realizar la sustitución
total_conteo_pachuca.loc[total_conteo_pachuca['año'] == "2024", 'num_registros'] = 7884

# Verifica el cambio en el DataFrame
print(total_conteo_pachuca)


    año  num_registros         modalidad
0  2025           9264  Créditos (todos)
1  2026           9288  Créditos (todos)
2  2019           8676  Créditos (todos)
3  2020           8920  Créditos (todos)
4  2021          10112  Créditos (todos)
5  2022           8873  Créditos (todos)
6  2023           8364  Créditos (todos)
7  2024           7884  Créditos (todos)


### Modalidad 1: Nueva

In [22]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y contar el número de registros únicos por modalidad en cada año
    conteo_creditos_año = df.groupby('modalidad').size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    conteo_creditos_año['año'] = año
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,num_registros,año
0,1,4903,2019
1,2,1186,2019
2,3,2092,2019
3,4,495,2019
4,1,4915,2020
5,2,1082,2020
6,3,2143,2020
7,4,780,2020
8,1,5082,2021
9,2,1301,2021


In [23]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y contar el número de registros
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año']).size().reset_index(name='num_registros')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes    1    2    3    4    5    6    7    8    9    10   11   12
 año                                                             
 2019  327  432  409  329  388  416  445  416  386  415  389  551
 2020  345  416  448  329  351  443  399  366  441  425  398  554
 2021  340  415  465  417  455  390  452  419  414  419  388  508
 2022  334  391  400  356  442  458  409  392  390  365  372  473
 2023  311  288  417  339  370  372  316  351  366  375  380  465
 2024  425  435  416  484  427  447    0    0    0    0    0    0,
 mes    1    2    3    4    5    6    7    8    9    10   11   12
 año                                                             
 2019   90  137  197  172  172  181  180  178  187  191  185  222
 2020   96  172  199  119  119  156  156  173  207  259  233  254
 2021  111  183  242  201  234  302  209  197  207  211  224  256
 2022  106  159  208  159  204  212  185  197  197  189  175  242
 2023  122  202  220  182  182  224  197  210  207  223  202  265
 2024  21

In [24]:
X = df_modalidad_1[['año', 'mes']]
y = df_modalidad_1['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [25]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024,6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 


# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([df_modalidad_1, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca['modalidad'] = 'Nueva'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo_pachuca['num_registros'] = nueva_conteo_pachuca['num_registros'].map(format_number)
nueva_conteo_pachuca

,año,num_registros,modalidad
0,2019,"4,903",Nueva
1,2020,"4,915",Nueva
2,2021,"5,082",Nueva
3,2022,"4,782",Nueva
4,2023,"4,350",Nueva
5,2024,"5,058",Nueva
6,2025,"4,576",Nueva
7,2026,"4,500",Nueva


In [26]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [27]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 


# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,num_registros,modalidad
0,2019,"2,092",Usada
1,2020,"2,143",Usada
2,2021,"2,577",Usada
3,2022,"2,233",Usada
4,2023,"2,436",Usada
5,2024,"3,160",Usada
6,2025,"2,825",Usada
7,2026,"2,938",Usada


In [28]:
conteo_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [29]:
# Convertir 'num_registros' a enteros y eliminar comas
conteo_pachuca['num_registros'] = conteo_pachuca['num_registros'].apply(lambda x: int(str(x).replace(',', '')))

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año  num_registros         modalidad
2  2019           8676  Créditos (todos)
0  2019           4903             Nueva
0  2019           2092             Usada
1  2020           4915             Nueva
1  2020           2143             Usada
3  2020           8920  Créditos (todos)
4  2021          10112  Créditos (todos)
2  2021           5082             Nueva
2  2021           2577             Usada
3  2022           2233             Usada
3  2022           4782             Nueva
5  2022           8873  Créditos (todos)
4  2023           4350             Nueva
6  2023           8364  Créditos (todos)
4  2023           2436             Usada
7  2024           7884  Créditos (todos)
5  2024           5058             Nueva
5  2024           3160             Usada
0  2025           9264  Créditos (todos)
6  2025           4576             Nueva
6  2025           2825             Usada
1  2026           9288  Créditos (todos)
7  2026           4500             Nueva
7  2026         

In [30]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['num_registros'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['num_registros'],  # Añadir el texto de los valores
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    #yaxis_title="Número de Registros",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Configurar el eje Y para mostrar los valores completos en miles (sin 'K')
fig.update_yaxes(tickformat=",")  # ',' para formato de miles sin abreviación


# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_cantidad', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [TOTAL]**

In [19]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).sum().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_suma_creditos_pachuca

,año,mes,montos
0,2019,1,7.490206e+08
1,2019,2,1.037917e+09
2,2019,3,1.177932e+09
3,2019,4,1.030133e+09
4,2019,5,1.239560e+09
...,...,...,...
61,2024,2,2.082487e+09
62,2024,3,1.987884e+09
63,2024,4,2.336097e+09
64,2024,5,2.020927e+09


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_suma_creditos_pachuca[['año', 'mes']]
y = total_suma_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [33]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
 #                                 'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_suma_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)

total_todos_pachuca = total_todos_pachuca.iloc[1:].reset_index(drop=True)

total_todos_pachuca

,año,montos,modalidad
0,2025,"26,782,488,993.00",Créditos (todos)
1,2026,"28,632,328,690.07",Créditos (todos)
2,2019,"15,004,579,763.63",Créditos (todos)
3,2020,"16,244,753,217.27",Créditos (todos)
4,2021,"21,181,132,336.27",Créditos (todos)
5,2022,"21,761,177,351.19",Créditos (todos)
6,2023,"21,311,155,047.85",Créditos (todos)
7,2024,"12,267,131,712.70",Créditos (todos)


In [34]:
# Valores conocidos
registros_2022 = 21761177351
registros_2023 = 21311155047

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))

Estimación de registros en 2024: 20870439228


In [35]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Ahora intenta realizar la sustitución
total_todos_pachuca.loc[total_todos_pachuca['año'] == "2024", 'montos'] = 20870439228


# Verifica el cambio en el DataFrame
print(total_todos_pachuca)


    año       montos         modalidad
0  2025  26782488993  Créditos (todos)
1  2026  28632328690  Créditos (todos)
2  2019  15004579763  Créditos (todos)
3  2020  16244753217  Créditos (todos)
4  2021  21181132336  Créditos (todos)
5  2022  21761177351  Créditos (todos)
6  2023  21311155047  Créditos (todos)
7  2024  20870439228  Créditos (todos)


In [36]:
df_financiamientos_2024_queretaro.shape

(5435, 17)

In [37]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].sum().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, suma_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,montos,año
0,1,1.151847e+10,2019
1,2,1.269736e+08,2019
2,3,2.714055e+09,2019
3,4,6.450812e+08,2019
4,1,1.146007e+10,2020
5,2,3.190812e+08,2020
6,3,3.112169e+09,2020
7,4,1.353432e+09,2020
8,1,1.415682e+10,2021
9,2,2.807882e+08,2021


In [38]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].sum().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3             4             5   \
 año                                                                          
 2019  6.113629e+08  8.249966e+08  8.795315e+08  7.559109e+08  9.573430e+08   
 2020  7.494082e+08  8.384365e+08  9.744272e+08  6.053918e+08  7.612161e+08   
 2021  7.017406e+08  1.135347e+09  1.347438e+09  1.098155e+09  1.209389e+09   
 2022  8.027088e+08  1.177818e+09  1.260772e+09  9.787978e+08  1.455482e+09   
 2023  8.973257e+08  9.192939e+08  1.310117e+09  9.539358e+08  1.056246e+09   
 2024  1.245438e+09  1.224518e+09  1.056756e+09  1.350014e+09  1.227256e+09   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  1.040966e+09  1.091712e+09  8.910569e+08  8.690005e+08  1.022977e+09   
 2020  1.020258e+09  9.408024e+08  8.872832e+08  1.135721e+09  9.515098e+08   
 2021  1.028392e+09  1.147071e+09  1.117317e+09  1

In [39]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]

In [40]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo_pachuca = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].sum().reset_index()

# Mostrar el DataFrame resultante
print(nueva_conteo_pachuca)

    año modalidad        montos
0  2019     Nueva  1.151847e+10
1  2020     Nueva  1.146007e+10
2  2021     Nueva  1.415682e+10
3  2022     Nueva  1.525266e+10
4  2023     Nueva  1.403260e+10
5  2024     Nueva  1.622326e+10
6  2025     Nueva  1.700771e+10
7  2026     Nueva  1.793164e+10


In [41]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)


LinearRegression()

In [42]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
#def format_number(x):
 #   return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
#mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,montos,modalidad
0,2019,2.714055e+09,Usada
1,2020,3.112169e+09,Usada
2,2021,4.220520e+09,Usada
3,2022,4.231726e+09,Usada
4,2023,5.600430e+09,Usada
5,2024,7.290758e+09,Usada
6,2025,7.090518e+09,Usada
7,2026,7.840256e+09,Usada


In [43]:
conteo_pachuca = pd.concat([
    total_todos_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [44]:
conteo_pachuca

,año,montos,modalidad
0,2025,2.678249e+10,Créditos (todos)
1,2026,2.863233e+10,Créditos (todos)
2,2019,1.500458e+10,Créditos (todos)
3,2020,1.624475e+10,Créditos (todos)
4,2021,2.118113e+10,Créditos (todos)
5,2022,2.176118e+10,Créditos (todos)
6,2023,2.131116e+10,Créditos (todos)
7,2024,2.087044e+10,Créditos (todos)
0,2019,1.151847e+10,Nueva
1,2020,1.146007e+10,Nueva


In [45]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año       montos         modalidad
2  2019  15004579763  Créditos (todos)
0  2019  11518470288             Nueva
0  2019   2714054711             Usada
1  2020  11460071273             Nueva
1  2020   3112169040             Usada
3  2020  16244753217  Créditos (todos)
4  2021  21181132336  Créditos (todos)
2  2021  14156817374             Nueva
2  2021   4220520104             Usada
3  2022   4231725816             Usada
3  2022  15252659268             Nueva
5  2022  21761177351  Créditos (todos)
4  2023  14032595647             Nueva
6  2023  21311155047  Créditos (todos)
4  2023   5600430244             Usada
7  2024  20870439228  Créditos (todos)
5  2024  16223259200             Nueva
5  2024   7290757756             Usada
0  2025  26782488993  Créditos (todos)
6  2025  17007714798             Nueva
6  2025   7090517757             Usada
1  2026  28632328690  Créditos (todos)
7  2026  17931639358             Nueva
7  2026   7840256461             Usada


In [57]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto total anual",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)

# Configurar el eje Y utilizando el formateo personalizado
tickvals = [1_000, 1_000_000, 1_000_000_000,5_000_000_000,10_000_000_000,15_000_000_000,20_000_000_000,25_000_000_000, 30_000_000_000]  # Valores hasta 30 mil millones
ticktext = [formatear_numero(val) for val in tickvals]  # Formatea cada valor en palabras

fig.update_yaxes(
    tickvals=tickvals, 
    ticktext=ticktext
)

# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_total', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [PROMEDIO]**

In [8]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).mean().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos = pd.concat(dfs, ignore_index=True)
total_suma_creditos

,año,mes,montos
0,2019,1,1.347159e+06
1,2019,2,1.396928e+06
2,2019,3,1.521875e+06
3,2019,4,1.640339e+06
4,2019,5,2.032065e+06
...,...,...,...
61,2024,2,2.222505e+06
62,2024,3,2.170179e+06
63,2024,4,2.329110e+06
64,2024,5,2.301739e+06


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Preparar datos para el modelo
X = total_suma_creditos[['año', 'mes']]
y = total_suma_creditos['montos']
# Normalizar los datos
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [10]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_suma_creditos, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
total_todos_pachuca = total_todos_pachuca.iloc[1:].reset_index(drop=True)

total_todos_pachuca

c:\Users\yoe11\anaconda3\envs\newst\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


,año,montos,modalidad
0,2025,"2,065,446,348.76",Créditos (todos)
1,2026,"2,066,463,795.18",Créditos (todos)
2,2019,"1,715,876.99",Créditos (todos)
3,2020,"1,790,039.91",Créditos (todos)
4,2021,"2,078,878.44",Créditos (todos)
5,2022,"2,435,874.90",Créditos (todos)
6,2023,"2,533,385.99",Créditos (todos)
7,2024,"2,255,251.93",Créditos (todos)


In [12]:
# Valores conocidos
registros_2022 = 2435874.90
registros_2023 = 2533385.99

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024, 2025, y 2026 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)
estimacion_2025 = estimacion_2024 * (1 + tasa_cambio)
estimacion_2026 = estimacion_2025 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))
print("Estimación de registros en 2025:", round(estimacion_2025))
print("Estimación de registros en 2026:", round(estimacion_2026))

Estimación de registros en 2024: 2634801
Estimación de registros en 2025: 2740275
Estimación de registros en 2026: 2849971


In [6]:
round(df_financiamientos_2024_queretaro['monto'].mean(),0)

np.float64(2257062.0)

In [13]:
# Remover comas y convertir la columna 'montos' a enteros
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Ahora intenta realizar la sustitución
total_todos_pachuca.loc[total_todos_pachuca['año'] == "2024", 'montos'] = 2634801
total_todos_pachuca.loc[total_todos_pachuca['año'] == 2025, 'montos'] = 2740275
total_todos_pachuca.loc[total_todos_pachuca['año'] == 2026, 'montos'] = 2849971
total_todos = total_todos_pachuca.copy()
# Verifica el cambio en el DataFrame
print(total_todos)


    año   montos         modalidad
0  2025  2740275  Créditos (todos)
1  2026  2849971  Créditos (todos)
2  2019  1715876  Créditos (todos)
3  2020  1790039  Créditos (todos)
4  2021  2078878  Créditos (todos)
5  2022  2435874  Créditos (todos)
6  2023  2533385  Créditos (todos)
7  2024  2634801  Créditos (todos)


In [14]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].mean().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, suma_creditos_año], ignore_index=True)

total_conteo_creditos

,modalidad,montos,año
0,1,2.349270e+06,2019
1,2,1.070603e+05,2019
2,3,1.297349e+06,2019
3,4,1.303194e+06,2019
4,1,2.331652e+06,2020
5,2,2.948994e+05,2020
6,3,1.452249e+06,2020
7,4,1.735169e+06,2020
8,1,2.785678e+06,2021
9,2,2.158249e+05,2021


In [15]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].mean().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3             4             5   \
 año                                                                          
 2019  1.869611e+06  1.909714e+06  2.150444e+06  2.297601e+06  2.467379e+06   
 2020  2.172198e+06  2.015472e+06  2.175061e+06  1.840097e+06  2.168707e+06   
 2021  2.063943e+06  2.735776e+06  2.897717e+06  2.633466e+06  2.657997e+06   
 2022  2.403320e+06  3.012322e+06  3.151930e+06  2.749432e+06  3.292946e+06   
 2023  2.885292e+06  3.191993e+06  3.141768e+06  2.813970e+06  2.854718e+06   
 2024  2.930443e+06  2.814985e+06  2.540279e+06  2.789285e+06  2.874137e+06   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  2.502322e+06  2.453286e+06  2.141964e+06  2.251297e+06  2.465005e+06   
 2020  2.303067e+06  2.357901e+06  2.424271e+06  2.575331e+06  2.238847e+06   
 2021  2.636901e+06  2.537768e+06  2.666627e+06  3

In [16]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]
modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17392\3999403270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


In [17]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].mean().reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo['montos'] = nueva_conteo['montos'].map(format_number)

# Mostrar el DataFrame resultante
print(nueva_conteo)

    año modalidad   montos
0  2019     Nueva  2323536
1  2020     Nueva  2299039
2  2021     Nueva  2763776
3  2022     Nueva  3170567
4  2023     Nueva  3207899
5  2024     Nueva  3189465
6  2025     Nueva  3572015
7  2026     Nueva  3785244


In [18]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [19]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo = mejoramiento_conteo.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo['montos'] = mejoramiento_conteo['montos'].map(format_number)
mejoramiento_conteo

,año,montos,modalidad
0,2019,1285994,Usada
1,2020,1394197,Usada
2,2021,1602381,Usada
3,2022,1873857,Usada
4,2023,2273678,Usada
5,2024,2353707,Usada
6,2025,2607125,Usada
7,2026,2839432,Usada


In [14]:
conteo_pachuca = pd.concat([
    total_todos,
    nueva_conteo,
    mejoramiento_conteo,
])

In [21]:
conteo_pachuca = conteo_pachuca.round(2)
conteo_pachuca

,año,montos,modalidad
0,2025,2740275,Créditos (todos)
1,2026,2849971,Créditos (todos)
2,2019,1715876,Créditos (todos)
3,2020,1790039,Créditos (todos)
4,2021,2078878,Créditos (todos)
5,2022,2435874,Créditos (todos)
6,2023,2533385,Créditos (todos)
7,2024,2634801,Créditos (todos)
0,2019,2323536,Nueva
1,2020,2299039,Nueva


In [22]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año   montos         modalidad
2  2019  1715876  Créditos (todos)
0  2019  2323536             Nueva
0  2019  1285994             Usada
1  2020  2299039             Nueva
1  2020  1394197             Usada
3  2020  1790039  Créditos (todos)
4  2021  2078878  Créditos (todos)
2  2021  2763776             Nueva
2  2021  1602381             Usada
3  2022  1873857             Usada
3  2022  3170567             Nueva
5  2022  2435874  Créditos (todos)
4  2023  3207899             Nueva
6  2023  2533385  Créditos (todos)
4  2023  2273678             Usada
7  2024  2634801  Créditos (todos)
5  2024  3189465             Nueva
5  2024  2353707             Usada
0  2025  2740275  Créditos (todos)
6  2025  3572015             Nueva
6  2025  2607125             Usada
1  2026  2849971  Créditos (todos)
7  2026  3785244             Nueva
7  2026  2839432             Usada


In [36]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto promedio anual",
    yaxis_tickformat=",.0f",  # Formatear el eje y con separadores de coma y sin decimales
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_promedio', carpeta='assets/graficas')
fig.show()

# ff

# **3.4 Forecast Monto de créditos generales. [PROMEDIO]**

In [2]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).mean().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos = pd.concat(dfs, ignore_index=True)
total_suma_creditos

,año,mes,montos
0,2019,1,1.347159e+06
1,2019,2,1.396928e+06
2,2019,3,1.521875e+06
3,2019,4,1.640339e+06
4,2019,5,2.032065e+06
...,...,...,...
61,2024,2,2.222505e+06
62,2024,3,2.170179e+06
63,2024,4,2.329110e+06
64,2024,5,2.301739e+06


In [3]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].mean().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, suma_creditos_año], ignore_index=True)

total_conteo_creditos

,modalidad,montos,año
0,1,2.349270e+06,2019
1,2,1.070603e+05,2019
2,3,1.297349e+06,2019
3,4,1.303194e+06,2019
4,1,2.331652e+06,2020
5,2,2.948994e+05,2020
6,3,1.452249e+06,2020
7,4,1.735169e+06,2020
8,1,2.785678e+06,2021
9,2,2.158249e+05,2021


In [4]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].mean().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3             4             5   \
 año                                                                          
 2019  1.869611e+06  1.909714e+06  2.150444e+06  2.297601e+06  2.467379e+06   
 2020  2.172198e+06  2.015472e+06  2.175061e+06  1.840097e+06  2.168707e+06   
 2021  2.063943e+06  2.735776e+06  2.897717e+06  2.633466e+06  2.657997e+06   
 2022  2.403320e+06  3.012322e+06  3.151930e+06  2.749432e+06  3.292946e+06   
 2023  2.885292e+06  3.191993e+06  3.141768e+06  2.813970e+06  2.854718e+06   
 2024  2.930443e+06  2.814985e+06  2.540279e+06  2.789285e+06  2.874137e+06   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  2.502322e+06  2.453286e+06  2.141964e+06  2.251297e+06  2.465005e+06   
 2020  2.303067e+06  2.357901e+06  2.424271e+06  2.575331e+06  2.238847e+06   
 2021  2.636901e+06  2.537768e+06  2.666627e+06  3

In [5]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]
modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_2408\3999403270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = modalidad1[['año', 'mes']]
y = modalidad1['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [7]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].mean().reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo['montos'] = nueva_conteo['montos'].map(format_number)

# Mostrar el DataFrame resultante
print(nueva_conteo)

    año modalidad   montos
0  2019     Nueva  2323536
1  2020     Nueva  2299039
2  2021     Nueva  2763776
3  2022     Nueva  3170567
4  2023     Nueva  3207899
5  2024     Nueva  3189465
6  2025     Nueva  3572015
7  2026     Nueva  3785244


In [8]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [9]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo = mejoramiento_conteo.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo['montos'] = mejoramiento_conteo['montos'].map(format_number)
mejoramiento_conteo

,año,montos,modalidad
0,2019,1285994,Usada
1,2020,1394197,Usada
2,2021,1602381,Usada
3,2022,1873857,Usada
4,2023,2273678,Usada
5,2024,2353707,Usada
6,2025,2607125,Usada
7,2026,2839432,Usada


In [10]:
conteo_pachuca = pd.concat([
    #total_todos,
    nueva_conteo,
    mejoramiento_conteo,
])

In [11]:
conteo_pachuca = conteo_pachuca.round(2)
conteo_pachuca

,año,modalidad,montos
0,2019,Nueva,2323536
1,2020,Nueva,2299039
2,2021,Nueva,2763776
3,2022,Nueva,3170567
4,2023,Nueva,3207899
5,2024,Nueva,3189465
6,2025,Nueva,3572015
7,2026,Nueva,3785244
0,2019,Usada,1285994
1,2020,Usada,1394197


In [12]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año modalidad   montos
0  2019     Nueva  2323536
0  2019     Usada  1285994
1  2020     Usada  1394197
1  2020     Nueva  2299039
2  2021     Nueva  2763776
2  2021     Usada  1602381
3  2022     Usada  1873857
3  2022     Nueva  3170567
4  2023     Usada  2273678
4  2023     Nueva  3207899
5  2024     Nueva  3189465
5  2024     Usada  2353707
6  2025     Usada  2607125
6  2025     Nueva  3572015
7  2026     Nueva  3785244
7  2026     Usada  2839432


In [13]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
modalidades = conteo_pachuca['modalidad'].unique()
#colores = ['red', 'lightgreen']  # Definir los colores a usar
for i, modalidad in enumerate(modalidades):
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center",  # Posicionar el texto encima de los puntos
        #line=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
        #marker=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto promedio anual",
    yaxis_tickformat=",.0f",  # Formatear el eje y con separadores de coma y sin decimales
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_promedio', carpeta='assets/graficas')
fig.show()